## Data Ingestion

In [5]:
import os

%pwd

'd:\\[1] Study\\[3] Data Science\\[6] Projects\\Text-Summarization-App\\research'

In [6]:
# I need to go back as of now to get to the root folder

os.chdir("../")

%pwd

'd:\\[1] Study\\[3] Data Science\\[6] Projects\\Text-Summarization-App'

In [7]:
# Entity is written type of the function
# Entity Creation for data ingestion

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class Data_Ingestion_Config:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [8]:
# Updating the configuration manager

from textSummarizerApp.constants import *
from textSummarizerApp.utils.common import read_yaml, create_directories


class Configuration_Manager:
    def __init__(self, config_path = str(CONFIG_FILE_PATH),
        params_path = str(PARAMS_FILE_PATH)):
        
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> Data_Ingestion_Config:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = Data_Ingestion_Config(
            root_dir = config.root_dir,
            source_url = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

In [11]:
# Update the components

import os
import urllib.request as request
import zipfile
from textSummarizerApp.utils.common import get_size
from textSummarizerApp.logging import logger

class Data_Ingestion:
    def __init__(self, config: Data_Ingestion_Config):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f"Downloaded file {filename} with information \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")


    def unzip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [12]:
# Creating the pipline
# First need to initialize the configuration manager
# After ingesting the data, 


try:
    config = Configuration_Manager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = Data_Ingestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.unzip_file()
except Exception as e:
    raise e

[2023-09-18 21:10:54,598: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-09-18 21:10:54,600: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-18 21:10:54,601: INFO: common: Directory created at artifacts]
[2023-09-18 21:10:54,602: INFO: common: Directory created at artifacts/data_ingestion]
[2023-09-18 21:11:06,951: INFO: 133072220: Downloaded file artifacts/data_ingestion/samsum_data.zip with information 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 3AA8:0CF6:790B45:93C824:65087693
Accept-Ranges: bytes
Date: Mon, 18 Sep 2023 16:11:03 GMT
Via: 1.1 varnish
X-Served-By: cache-fjr990026-FJR
X-Cac